In [1]:
# This file is the Jupyter notebook for the Machine Learning 1 Lab 1
# Author: Tom Pengilly
# Group Members: Tom Pengilly, Quynh Chao, Michael Weatherford, Anish Patel
# Date: 9/3/2020


In [3]:
########## Business Understanding (This Section has no code)

# What is the purpose of the data? (WHy was it collected?)
# How would you define and measure the outcomes from the dataset.... 
# ....(Why is the data important and how do you know if you have mined useful knowledge from the dataset?)
# How would you measure the effectiveness of a good prediction algorithm? Be specific.

Data Understanding: The following section includes the code used to aggregate the 5 data files into 1 combined dataset, data wrangling, missing/outlier handling, imputation, visualization, attribute relations, and interesting findings.

Data Wrangling: The data wrangling code used to concatenate, clean, and convert our data is shown below.  The first section deals with creating our final dataset for use in our analysis.

In [ ]:
########## Data Understanding ########## 

# Data preprocessing

In [87]:
# Import the required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import set_option
set_option('display.max_columns',400)
from pandas_profiling import ProfileReport
import copy as cp
import os
import glob
import matplotlib as plt
import re
import datetime
from scipy.stats import ks_2samp, ttest_ind
# from pandas.tools.plotting import scatter_matrix # #??? Need this?
# from pandas.tools.plotting import parallel_coordinates
# import seaborn as sns

In [ ]:
# Concatenate all data files and create the combined dataset
os.chdir('C:\\Users\\Tpeng\\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\DatasetAndPreprocessing')
path = os.getcwd()
print(path)

#Read and Concat all data files (excluding the combined dataset created later stored here)
all_files = glob.glob(path + "/*.csv")
i = 0

for obs in all_files:
    if re.search('Combined_Dataset', obs):
        print(i)
        i += 1
        all_files.remove(obs)
    else:
        i += 1
        
property_data = []

# Create the combined dataset

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    property_data.append(df)

combineddf = pd.concat(property_data, axis=0, ignore_index=True, sort = False)

In [ ]:
# Reformat attributes in order to explore/change them more easily
# Split variables into ordinal, continuous and categorical
ordinal_vars = ['rooms', 'bedrooms', 'bathrooms' ]
continuous_vars = ['lat', 'lon', 'surface_total', 'surface_covered', 'price']
categorical_vars = ['ad_type', 'l1', 'l2', 'l3', 'l4', 'l5', 'l6', 'currency', 'price_period', 'property_type', 'operation_type']
string_vars = ['id', 'title', 'description']
time_vars = ['start_date', 'end_date', 'created_on']

# Create a dataframe with  missing values as -1 or 0 (this is done to allow the datatypes to be changed)
# dates of 9999-12-31 have not yet expired, but are replaced w/ 1970-01-01 since this value is used to determine exchange rates
combineddf[ordinal_vars] = combineddf[ordinal_vars].replace(to_replace = np.nan, value = -1)
combineddf[time_vars] = combineddf[time_vars].replace(to_replace = "9999-12-31", value = 0)

# Change data types
combineddf[ordinal_vars] = combineddf[ordinal_vars].astype(np.int64)
combineddf[continuous_vars] = combineddf[continuous_vars].astype(np.float64)
combineddf[categorical_vars] = combineddf[categorical_vars].astype('category')
combineddf[string_vars] = combineddf[string_vars].astype(str)
combineddf[time_vars] = pd.to_datetime(combineddf[time_vars].stack(), format = "%Y-%m-%d").unstack()

In [ ]:
# Remove listings that have not yet expired (end date 9999-12-31)
indices = []
for row in range(len(combineddf)):
    if combineddf.end_date[row].strftime('%Y-%m-%d') == '1970-01-01':
        indices.append(row)
reduceddf = combineddf.drop(combineddf.index[indices], inplace = False)
reduceddf = reduceddf.reset_index(drop=True)

# Remove listings that have no price or currency
indices = []
for row in range(len(reduceddf)):
    if reduceddf.price[row] == 0:
        indices.append(row)
    if pd.isnull(reduceddf.price[row]):
        indices.append(row)
    if pd.isnull(reduceddf.currency[row]):
        indices.append(row)
reduceddf2 = reduceddf.drop(combineddf.index[indices], inplace = False)
reduceddf2 = reduceddf2.reset_index(drop=True)

# Combine factor levels that are equivalent
reduceddf2 = reduceddf2.replace(to_replace = 'Estados Unidos de América', value = 'Estado Unidos')

The following code is used to read in historical exchange rate data between all currencies and USD. This data is used to convert all prices to the same scale. The transaction end_date was used as the date of exchange, if the date isn't listed in the price histories, the date is decremented until a recent exchange rate was found.

In [4]:
# Convert prices to USD using exchange rate histories.  All price histories from https://www.investing.com/currencies/
# The exchange rate for the given currency pair is looked up for the end_date
arsusd = pd.read_csv('C:\\Users\\Tpeng\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\Price Data\\USD_ARS Historical Data.csv', header = 0)
copusd = pd.read_csv('C:\\Users\\Tpeng\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\Price Data\\USD_COP Historical Data.csv', header = 0)
penusd = pd.read_csv('C:\\Users\\Tpeng\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\Price Data\\USD_PEN Historical Data.csv', header = 0)
uyuusd = pd.read_csv('C:\\Users\\Tpeng\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\Price Data\\USD_UYU Historical Data.csv', header = 0)


# Convert price in reduceddf to USD 
for obs in range(len(reduceddf2)):
    if reduceddf2.currency[obs] != 'USD':
        currency = reduceddf2.currency[obs]
        date = reduceddf2.end_date[obs]
        
        if date.strftime('%Y-%m-%d') != '1970-01-01':
            if reduceddf2.currency[obs] == 'ARS':
                while date.strftime('%Y-%m-%d') not in list(arsusd.Date):
                    date = date - datetime.timedelta(days = 1)
                exchange_rate = arsusd.Price[arsusd.Date == date.strftime('%Y-%m-%d')]
                reduceddf2.at[obs, 'price'] = reduceddf2.price[obs] / float(exchange_rate)
                reduceddf2.at[obs, 'currency'] = 'USD'
            if reduceddf2.currency[obs] == 'UYU':
                while date.strftime('%Y-%m-%d') not in list(uyuusd.Date):
                    date -= datetime.timedelta(days = 1)
                exchange_rate = uyuusd.Price[uyuusd.Date == date.strftime('%Y-%m-%d')]
                reduceddf2.at[obs, 'price'] = reduceddf2.price[obs] / float(exchange_rate)
                reduceddf2.at[obs, 'currency'] = 'USD'
            if reduceddf2.currency[obs] == 'PEN':
                while date.strftime('%Y-%m-%d') not in list(penusd.Date):
                    date -= datetime.timedelta(days = 1)
                exchange_rate = penusd.Price[penusd.Date == date.strftime('%Y-%m-%d')]
                reduceddf2.at[obs, 'price'] = reduceddf2.price[obs] / float(exchange_rate)
                reduceddf2.at[obs, 'currency'] = 'USD'
            if reduceddf2.currency[obs] == 'COP':
                while date.strftime('%Y-%m-%d') not in list(copusd.Date):
                    date -= datetime.timedelta(days = 1)
                exchange_rate = copusd.Price[copusd.Date == date.strftime('%Y-%m-%d')]
                reduceddf2.at[obs, 'price'] = reduceddf2.price[obs] / float(exchange_rate)
                reduceddf2.at[obs, 'currency'] = 'USD'
                
# The dataset is saved as a new file at this point
#reduceddf2.to_csv("Converted_Dataset.csv", sep = ',')
#converted_data = pd.read_csv('C:\\Users\\Tpeng\\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\Datasets\\Converted_Dataset.csv', sep = ',', header = 0)
converted_data = reduceddf2

NameError: name 'reduceddf2' is not defined

In [ ]:
# Create dataframe filled with only missing values to determine appropriate action
missing_dates = converted_data[converted_data.start_date.isna() & converted_data.end_date.isna() & converted_data.created_on.isna()]

# These 65 observations are missing all dates, countries, prices, currency, etc... remove these
converted_data2 = converted_data[~converted_data['Unnamed: 0'].isin(missing_dates['Unnamed: 0'])]

# Reset indices on converted_data2
converted_data2 = converted_data2.reset_index(drop=True)

# Rename index column
converted_data2 = converted_data2.rename(columns={'Unnamed: 0': 'Index'})

# Replace innapropriate -1 values with np.nan and remove duplicates
converted_data2 = converted_data2.replace(to_replace = (-1, -2, -3), value = np.nan)
converted_data2.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
# Replace negative surface_total and surface_covered values with nan
converted_data2.surface_covered.replace(to_replace = -152, value = np.nan, inplace = True)
converted_data2.surface_covered.replace(to_replace = -4, value = np.nan, inplace = True)
converted_data2.surface_total.replace(to_replace = -36, value = np.nan, inplace = True)

In [14]:
###### FOR CONVENIENCE ####### Load the converted_data 
os.chdir('C:\\Users\\Tpeng\\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\')
path = os.getcwd()
converted_data2 = pd.read_csv('C:\\Users\\Tpeng\\OneDrive\\Documents\\SMU\\Term 3\\Machine Learning\\Lab1\\Datasets\\Converted_Dataset2.csv', sep = ',', header = 0)


C:\Users\Tpeng\Anaconda3\envs\renv\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
# Drop the extra index column and display header
converted_data2 = converted_data2.drop(['Unnamed: 0'], axis = 1)
converted_data2.head()

,Index,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
0,0,jgIzAKLaljBee5xKVoCs3A==,Propiedad,2019-09-15,2020-03-15,2019-09-15,-34.643029,-58.368769,Argentina,Capital Federal,Barracas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,556.438792,USD,Mensual,PH 5 AMB CON AMPLIA TERRAZA Y BALCÓN,CODIGO: 4429-11 ubicado en: AVENIDA PATRICIOS ...,PH,Alquiler
1,1,kzR5ghTwqnCfkf1A1CU6HA==,Propiedad,2019-09-15,2019-09-21,2019-09-15,-34.476510,-58.534146,Argentina,Bs.As. G.B.A. Zona Norte,San Isidro,San Isidro,NaN,NaN,4.0,NaN,2.0,110.0,NaN,441.150521,USD,Mensual,PH - Las Lomas-San Isidro,ALQUILER PH SIN EXPENSAS SAN ISIDRO Don Bosco...,PH,Alquiler
2,2,M60/Oh5ToxGELDNQHKASrQ==,Propiedad,2019-09-15,2019-10-29,2019-09-15,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,NaN,NaN,NaN,5.0,NaN,2.0,190.0,160.0,757.575758,USD,Mensual,Venta. Alquiler anual. Casa estilo moderno. 5...,"MAM (2). Hermosa casa diseñada en una planta, ...",Casa,Alquiler
3,3,pjPcCHWnjKcN05hNhI0ssg==,Propiedad,2019-09-15,2019-10-29,2019-09-15,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,NaN,NaN,NaN,5.0,NaN,2.0,190.0,160.0,757.575758,USD,Mensual,ALQUILER Casa estilo racionalista en una plan...,"MAM. Hermosa casa diseñada en una planta, con ...",Casa,Alquiler
4,4,0Py/8IGj4qZsfLk7lEgPvA==,Propiedad,2019-09-15,2019-09-17,2019-09-15,-31.431401,-64.225170,Argentina,Córdoba,Córdoba,NaN,NaN,NaN,2.0,NaN,1.0,40.0,40.0,123.915737,USD,NaN,Consultorio en alquiler,"Consultorio ideal para centro de estética, pos...",Oficina,Alquiler


In [121]:
# Remove empty property_type listings
prop_cats = ['Departamento',
 'Otro',
 'Casa',
 'Apartamento',
 'Lote',
 'Local comercial',
 'Oficina',
 'PH',
 'Depósito',
 'Finca',
 'Cochera',
 'Parqueadero',
 'Casa de campo',
 'Garaje']

final_df = converted_data2[converted_data2['property_type'].isin(prop_cats)]
props = final_df.property_type.astype('category')
final_df2 = final_df.assign(property_type = final_df['property_type'].astype('category'))

#final_df.drop(labels = 'property_type', axis = 'columns', inplace = True)

#final_df['property_type'] = final_df['property_type']

The following dataset is the *Final* dataset to be used for analysis

In [122]:
# Remove all non- South American Countries
countries = ['Argentina', 'Brasil', 'Columbia', 'Ecuador', 'Perú', 'Uruguay']
final_df2 = final_df2[final_df2['l1'].isin(countries)]
final_df2.l1 = final_df2['l1'].astype('category')
final_df3 = final_df2[final_df2.operation_type == 'Venta']

In [124]:
# Print to csv file
final_df3.to_csv("Final_Dataset.csv", sep = ',')

In [125]:
len(final_df3)

694993

In [123]:
final_df3


,Index,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,description,property_type,operation_type
24,24,EWeY8jVhb8ielLeKtfibVw==,Propiedad,2019-09-15,2020-01-20,2019-09-15,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,NaN,NaN,NaN,4.0,NaN,2.0,198.0,150.0,385000.0,USD,Mensual,"Venta. Casa en una planta, 4 amb. con galería ...",MAM.(2) Hermosa propiedad en Barrio San Franci...,Casa,Venta
25,25,IrEeG8ewIcfLVGSb14kH1w==,Propiedad,2019-09-15,2020-01-20,2019-09-15,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Norte,Tigre,NaN,NaN,NaN,4.0,NaN,2.0,198.0,150.0,385000.0,USD,Mensual,Venta. Casa en una planta.Lote interno. Barrio...,MAM. Hermosa propiedad en Barrio San Francisco...,Casa,Venta
26,26,9vIWv494LqC6crAkfpLeDg==,Propiedad,2019-09-15,2020-03-03,2019-09-15,NaN,NaN,Argentina,Santa Fe,Rosario,NaN,NaN,NaN,3.0,NaN,1.0,49.0,40.0,85000.0,USD,Mensual,Cabaña en la Isla sobre riacho Los Marinos - V...,Lote de 20 metros de frente por 130 metros de ...,Casa,Venta
27,27,aydrtgoZ/frzXDcV20McuA==,Propiedad,2019-09-15,2020-03-03,2019-09-15,-32.951146,-60.571979,Argentina,Entre Ríos,Victoria,NaN,NaN,NaN,3.0,NaN,1.0,49.0,40.0,85000.0,USD,Mensual,Cabaña en la Isla sobre riacho Los Marinos - V...,Lote de 20 metros de frente por 130 metros de ...,Casa,Venta
28,28,pcyMin/DYA9uU5PxtC+19w==,Propiedad,2019-09-15,2020-04-07,2019-09-15,-32.961060,-68.877991,Argentina,Mendoza,Luján de Cuyo,NaN,NaN,NaN,NaN,NaN,3.0,400.0,400.0,570000.0,USD,Mensual,Casa a Estrenar 4 Habitaciones - Diseño y Term...,WONDER VENDE Espectacular Casa a ESTRENAR de D...,Casa,Venta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1672195,1672195,yY75x3NqvRSMvb2Fj2KxUQ==,Propiedad,2019-03-26,2019-07-26,2019-03-26,-34.911168,-56.176738,Uruguay,Montevideo,Parque Rodo,NaN,NaN,NaN,NaN,5.0,2.0,NaN,450.0,320000.0,USD,NaN,OFICINAS PARA EMPRESA GRAN CASONA cw118747,ATENCIÓN OFICINAS Y EMPRESAS <br><br>CASONA TO...,Oficina,Venta
1672196,1672196,yq4yX7MERsET1joR2GKk+g==,Propiedad,2020-01-07,2020-03-05,2020-01-07,-34.906957,-56.206476,Uruguay,Montevideo,Ciudad Vieja,NaN,NaN,NaN,NaN,2.0,1.0,NaN,55.0,82000.0,USD,NaN,2 Dormitorio Ciudad Vieja cw135068,Ramar Real Estate ofrece Apartamento reciclado...,Departamento,Venta
1672197,1672197,Xz/K57XlxftCEVV42phoVw==,Propiedad,2020-01-07,2020-03-08,2020-01-07,-34.903460,-56.131983,Uruguay,Montevideo,NaN,NaN,NaN,NaN,3.0,2.0,2.0,NaN,87.0,355000.0,USD,NaN,"Puerto Buceo, 2 dorm, 2baños, 2 garajes",Contacto: Laura Castro Albernaz - MLS ID # 940...,Departamento,Venta
1672198,1672198,lPf9H1YHBunoKDRVO9Bzwg==,Propiedad,2020-01-07,2020-05-09,2020-01-07,-34.892173,-56.132236,Uruguay,Montevideo,Malvin,NaN,NaN,NaN,4.0,3.0,1.0,NaN,63.0,144800.0,USD,NaN,"Apto.3 dormitorios, al frente, luminoso.",Contacto: Esther Languillet - MLS ID # 9400410...,Departamento,Venta


In [ ]:
# Import dataset into R dataframe
Rdf = robjects.DataFrame

In [ ]:
# 1. Verify data quality.
# Explain and address missing values, duplicate data, and outliers. Are they mistakes? How do you deal with them? Be specific.
# Insert Preprocessing code here

In [5]:
# 2. Give appropriate statistics of the most important attributes in data
# Range, mode, mean, median, variance, counts, etc...
# Describe what they mean or if you found something interesting about them
# You can use data from other sources for comparison
# Explain significance of the statistics run and why they are meaningful



In [6]:
# 3. Visualize the most important attributes appropriately (AT LEAST 5 ATTRIBUTES)
# Provide an interpretation for each chart
# Explain each attribute why the chosen visualization is appropriate


In [7]:
# 4. Explain relationships b/t attributes
# Scatterplots, correlations, cross-tabulation, group-wise averages, etc. as appropriate and explain interesting relationships.


In [8]:
# 5. Identify and explain interesting relationships b/t features and the class you are trying to predict (ie. rel b/t
# vars and the target classification)


In [9]:
# 6. Are there other features that could be added to the data or created from existing features? Which ones?


In [ ]:
########## Exceptional Work ##########

# Free reign to provide additional analyses
# Idea: implement dimensionality reduction, then visualize and interpret the results

# Other Ideas: